# Notebook 3: Injection Generation

In this notebook, we will explore how to generate waveforms with GravyFlow, which can be used to approximate gravitational wave signals. Throughout the GravyFlow library, a significant amount of computation is performed on the GPU. This approach means that there may be some initial overheads compared to CPU calculations, so individual operations might perform more slowly. However, when used for thousands of iterations, GravyFlow methods are often orders of magnitude faster than the functions they were adapted from.

**Note:** As with the previous notebook, the iterators demonstrated in this notebook are not necessarily recommended for use in training machine learning models. Even if you only require waveforms, without any obfuscating noise, it is probably more convenient to use `gf.Dataset`, which will be explained in a later notebook. However, instances of `gf.Dataset` are composed by combining iterators of the type produced in this notebook with iterators shown in subsequent notebooks. Therefore, understanding the function of these iterators is useful.

We will begin by performing the necessary imports:

In [ ]:
# Built-in imports
from typing import List, Dict
from pathlib import Path

# Dependency imports: 
import numpy as np
import tensorflow as tf
from bokeh.io import show, output_notebook
from bokeh.layouts import gridplot

# Import the GravyFlow module.
import gravyflow as gf

As usual, we will also set up our environment:

**NOTE**: GravyFlow utilizes cuPhenom to generate IMRPhenomD waveforms on the GPU. It's important to note that cuPhenom will attempt to run on the same GPU as GravyFlow. However, cuPhenom does not use memory allocated for TensorFlow by the `gf.Env` function. Therefore, ensure that there is extra GPU memory available if you intend to generate IMRPhenomD waveforms with GravyFlow. This precaution helps in avoiding memory allocation issues and ensures smooth operation.

In [2]:
# Set up the environment using gf.env() and return a tf.distribute.Strategy object.
env : tf.distribute.Strategy = gf.env()

INFO:root:TensorFlow version: 2.15.0, CUDA version: 12.2


KeyboardInterrupt: 

## Generating Our First Waveform on the GPU

Currently, GravyFlow supports the generation of two waveform types: IMRPhenomD and White Noise Bursts (WNB).  We plan to expand this selection in the future. Both waveform types are generated on the GPU. IMRPhenomD generation utilizes the cuPhenom C library and operates using the CUDA library for GPU processing. On the other hand, WNB generation is integrated within GravyFlow and uses TensorFlow functions.

It's important to note that cuPhenom is designed exclusively for GPU use and does not have CPU support. As such, attempts to run it on a CPU will fail. Additionally, cuPhenom has independent error-checking systems, which may raise errors or cause crashes outside of GravyFlow's control. These errors, often Out Of Memory (OOM) issues, will typically be indicated in the error message. Future enhancements are aimed at more robust integration of cuPhenom errors and improved memory management.

Each waveform type has an associated generator class that inherits from a common parent: gf.WaveformGenerator. This parent has some attributes which are common to all waveform generators, but each waveform generator subclass has extra associated attributes dependent on the parameters of the waveform it is designed to generate. The following attributes are common to all waveform generators:

- `scaling_method`: `Union[gf.ScalingMethod, None]` = `None`
  > This parameter can accept a `gf.ScalingMethod` object, which can be used to scale this waveform when it is injected into obfuscating noise. This process will be discussed in more detail in a later notebook. When the default value, `None`, is used, no scaling will be applied.

- `injection_chance`: `float` = `1.0`
  > This parameter determines how likely it is for the waveform generator to return an injection vs a tensor of all zeros. This functionality is utilized when generating a dataset which contains a mix of examples with and without injected signals. We will also discuss the use of this parameter in more detail in future notebooks. The default value is 1.0, this will cause the iterator to generate injections on every iteration.

- `front_padding_duration_seconds`: `float` = `0.3`
  > GravyFlow Waveform generators output tensors which are the length of the onsource segment duration so that they can be easily summed with obfuscating noise. They will randomize the position of the waveform within that segment between front_padding_duration_seconds and back_padding_duration_seconds. In GravyFlow, injection padding is performed from the last sample of the generated waveform. For example, if front_padding_duration_seconds was 0.3 s, as is default, the last sample of the waveform cannot be less than 0.3 s from the start of the onsource segment. If the waveform is longer than the onsource duration, then the start of the waveform will be cropped and will not appear in the output tensor. If back_padding_duration_seconds was 0.3 s, then the last sample of the generated waveform cannot be within 0.3 s of the end of the onsource segment. If you wish to output waveforms in a fixed position with no random start time variance, set front_padding_duration_seconds = back_padding_duration_seconds = your desired last sample time in seconds. By default, front_padding_duration_seconds is 0.3 s.

- `back_padding_duration_seconds`: `float` = `0.0`
  > This parameter determines the closest the last sample of the generated waveform can be from the end of the onsource period. See the description of front_padding_duration_seconds for more clarification. The default value is 0.0 s, meaning that waveforms can generate up to the end of the onsource window.

- `scale_factor`: `Union[float, None]` = `None`
  > As has been mentioned previously, it is sometimes desirable to scale data to values close to one for easier consumption by artificial neural networks. Rather than using a form of normalization, GravyFlow introduces a scale factor which can be used to scale all waveforms by an identical amount. If None, which is the default, no scaling is performed.

- `network`: `Union[List[IFO], gf.Network, Path, None]` = `None`
  > We can also set the interferometer network configuration we wish to use to project this detector. This is not used by the waveform generator itself, but is used when this is used as part of a composition into a gf.Dataset, which will be described in later datasets. The default value is one.

Let's begin by generating a single IMRPhenomD waveform:

## Generating an IMRPhenomD Waveform

We can create an IMRPhenomD generator by initializing an instance of the `gf.cuPhenomDGenerator` class. This class includes all the parameters of the `gf.WaveformGenerator` class, listed above, along with these extra parameters which determine the properties of the IMRPhenomD waveform:

- `mass_1_msun` : `Union[float, gf.Distribution]` = `30.0`
  > Mass of companion one, in solar masses. The default value is 30.0 solar masses.

- `mass_2_msun` : `Union[float, gf.Distribution]` = `30.0`
  > Mass of companion two, in solar masses. The default value is 30.0 solar masses.

- `inclination_radians` : `Union[float, gf.Distribution]` = `0.0`
  > Inclination of the source in radians. The default is 0.0 radians.

- `distance_mpc` : `Union[float, gf.Distribution]` = `50.0`
  > Distance of the source in megaparsecs. The default value is 50.0 Mpc.

- `reference_orbital_phase_in` : `Union[float, gf.Distribution]` = `0.0`
  > Reference orbital phase in radians. The default is 0.0 radians.

- `ascending_node_longitude` : `Union[float, gf.Distribution]` = `0.0`
  > Longitude of ascending nodes. Degenerate with the polarization angle.

- `eccentricity` : `Union[float, gf.Distribution]` = `0.0`
  > Eccentricity at the reference epoch.

- `mean_periastron_anomaly` : `Union[float, gf.Distribution]` = `0.0`
  > Mean anomaly of periastron.

- `spin_1_in` : `Union[Tuple[float], gf.Distribution]` = `(0.0, 0.0, 0.0)`
  > Dimensionless spin components of the first companion.

- `spin_2_in` : `Union[Tuple[float], gf.Distribution]` = `(0.0, 0.0, 0.0)`
  > Dimensionless spin components of the second companion.

Let us create a PhenomD generator.

In [ ]:
# Initialize the PhenomD waveform generator:
# This generator specifically creates IMRPhenomD waveforms based on the given parameters.
phenom_d_generator: gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=50.0,          # Set the mass of the first companion to 50 solar masses.
    mass_2_msun=50.0,          # Set the mass of the second companion to 50 solar masses.
    inclination_radians=0.0    # Set the inclination of the source to 0 radians.
)

We can use the previously created waveform generator to compose a generic injection generator, which is an instance of `gf.InjectionGenerator`. The rationale behind this structure will become clearer as you further explore and work with the library.

Here are the parameters of `gf.InjectionGenerator`:

- `waveform_generators` : `Union[List[gf.WaveformGenerator], Dict[str, gf.WaveformGenerator]]`, Required
  > A list or dictionary of `gf.WaveformGenerator` instances to use for generating the injection. This is a required input, ensuring the injection generator has the necessary waveform data.

- `parameters_to_return` : `Union[List[WaveformParameters], None]` = `None`
  > A list of parameters to output from the injection generator. The default is None, meaning no extra waveform parameters will be returned unless specified.

- `seed` : `int` = `None`
  > The seed for the random number generators used in various aspects of the injection process, such as positioning the injection, generating injection parameters, and adding random elements to some injections like White Noise Bursts (WNBs). If set to None, the seed from `gf.Defaults` will be used.

Let's proceed to create an injection generator using only the `gf.cuPhenomDWaveformGenerator`:

In [ ]:
# Create an instance of the InjectionGenerator using the PhenomD waveform generator.
# This setup will enable the generation of injections based on the IMRPhenomD waveforms.
phenom_d_injection_generator: gf.InjectionGenerator = gf.InjectionGenerator(phenom_d_generator)

We can now utilize the `phenom_d_injection_generator` to rapidly generate IMRPhenomD waveforms on the GPU:

In [ ]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    # Generate a batch of IMRPhenomD waveforms using the phenom_d_injection_generator.
    # The num_examples_per_batch is set to 1 to limit the batch size to a single example.
    phenom_d_injection, _, _ = next(phenom_d_injection_generator(num_examples_per_batch=1))

    # The second and third returned variables will be discussed later in the notebook.

We can visualize the generated waveforms using the same plotting helper functions introduced in Notebook 2. It's important to note that the output of the injection generator matches the size of our onsource duration, plus two times our crop duration. This sizing ensures that the generated waveforms can be easily added to any background noise before undergoing the whitening process.

In [ ]:
# Generate strain plots for the Phenom D injection waveforms.
# The first index [0] selects the injection, the second index [0] selects the batch,
# and the third index [0] or [1] selects the polarization (plus or cross).
phenom_d_strain_plot = gf.generate_strain_plot(
    {
        "Plus Polarisation": phenom_d_injection[0][0][0],   # Plus polarization component of the waveform.
        "Cross Polarisation": phenom_d_injection[0][0][1]  # Cross polarization component of the waveform.
    },
    title="Phenom D Injection"
)

# Arrange the plot in a grid layout and display it in the notebook.
output_notebook()
show(phenom_d_strain_plot)

## Generating a White Noise Burst (WNB) on the GPU

In addition to generating IMRPhenomD waveforms, we can also create White Noise Bursts (WNB) on the GPU using the `gf.WNBGenerator`. This generator shares the parameters of the `gf.WaveformGenerator` class and includes additional parameters specific to WNBs:

- `duration_seconds` : `Union[float, gf.Distribution]` = `0.5`
  > Specifies the duration of the white noise burst in seconds. The default is set to 0.5 seconds, defining how long the burst will last.

- `min_frequency_hertz` : `Union[float, gf.Distribution]` = `16.0`
  > Determines the minimum frequency component to include in the WNB. The default setting is 16.0 Hertz, establishing the lower bound of the frequency range.

- `max_frequency_hertz` : `Union[float, gf.Distribution]` = `1024.0`
  > Sets the maximum frequency component for the WNB. The default is 1024.0 Hertz, indicating the upper limit of the frequency spectrum covered by the burst.

Let's proceed to create the `gf.WaveformGenerator` for generating a WNB:

In [ ]:
# Create a WaveformGenerator to generate White Noise Bursts (WNBs):
wnb_generator: gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=0.7,         # Set the duration of the WNB to 0.7 seconds.
    min_frequency_hertz=16.0,     # Set the minimum frequency of the WNB to 16.0 Hertz.
    max_frequency_hertz=1024.0    # Set the maximum frequency of the WNB to 1024.0 Hertz.
)

# Create an InjectionGenerator using the WNB waveform generator:
wnb_injection_generator: gf.InjectionGenerator = gf.InjectionGenerator(wnb_generator)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    # Generate a batch of injections with a single WNB injection:
    # The num_examples_per_batch is set to 1 to limit the batch size to one example.
    wnb_injection, _, _ = next(wnb_injection_generator(num_examples_per_batch=1))

Now that we have generated a White Noise Burst (WNB) using the `wnb_injection_generator`, let's proceed to visualize it. We'll use the same plotting functions as before to create a strain plot of the WNB:

In [ ]:
# Generate strain plots for the WNB injection waveforms.
# The indexing [0][0][0] and [0][0][1] access the plus and cross polarization components, respectively.
wnb_strain_plot = gf.generate_strain_plot(
    {
        "Plus Polarisation": wnb_injection[0][0][0],   # Plus polarization component of the WNB.
        "Cross Polarisation": wnb_injection[0][0][1]  # Cross polarization component of the WNB.
    },
    title="WNB Injection"
)

# Arrange the plot in a grid layout and display it in the notebook.
output_notebook()
show(wnb_strain_plot)

## Distributing Parameters

GravyFlow provides the `gf.Distribution` class to facilitate random variation of injection parameters for each example produced. This is especially useful when generating datasets to train gravitational wave models. The `gf.Distribution` class is preferred over distribution objects from libraries like SciPy and TensorFlow because it supports discrete and categorical distributions of arbitrary types, which are beneficial for hyperparameter optimization.

The supported distribution types include: `gf.DistributionType.CONSTANT`, `gf.DistributionType.UNIFORM`, `gf.DistributionType.NORMAL`, `gf.DistributionType.CHOICE`, `gf.DistributionType.LOG`, and `gf.DistributionType.POW_TWO`.

The arguments for `gf.Distribution` are as follows:

- `value` : `Union[int, float]` = `None`:
  > The specific value for the constant distribution. Only used when the distribution type is set to `gf.DistributionType.CONSTANT`.

- `dtype` : `Type` = `float`:
  > Data type of the distribution's output. Commonly set to float for waveform parameters.

- `min_` : `Union[int, float]` = `None`:
  > The minimum value for the distribution.

- `max_` : `Union[int, float]` = `None`:
  > The maximum value for the distribution.

- `mean` : `float` = `None`:
  > The mean (average) value for the distribution. Only used in the `gf.DistributionType.NORMAL` distribution type.

- `std` : `float` = `None`:
  > The standard deviation for the distribution, defining the spread of values. Only used in the `gf.DistributionType.NORMAL` distribution type.

- `possible_values` : `List` = `None`:
  > A list of possible values for the distribution. Essential for the `gf.DistributionType.CHOICE` distribution type, allowing specific values to be chosen randomly.

- `type_` : `gf.DistributionType` = `gf.DistributionType.CONSTANT`
  > The type of distribution to be used. This determines how the values will be generated and varied.

- `seed` : `int` = `None`:
  > Seed for the random number generator. Ensures reproducibility when generating random values. If `None`, the seed value from gf.Defaults will be used. Note that the seed will be reseeded using the parent object seed when `gf.Distributions` are used as part of an injection generator object.

In the next cell, we will demonstrate the use of `gf.Distributions`:

In [ ]:
# Create examples of different types of distributions using gf.Distribution
distribution_examples: Dict = {
    # Constant distribution with a fixed value
    "Constant": gf.Distribution(value=10, type_=gf.DistributionType.CONSTANT),

    # Uniform distribution with minimum and maximum values
    "Uniform": gf.Distribution(min_=5.0, max_=95.0, type_=gf.DistributionType.UNIFORM),

    # Normal distribution with specified mean and standard deviation
    "Normal": gf.Distribution(mean=0.0, std=3.0, type_=gf.DistributionType.NORMAL),

    # Choice distribution selecting randomly from a list of possible values
    "Choice": gf.Distribution(possible_values=["green", "red", "blue"], type_=gf.DistributionType.CHOICE),

    # Logarithmic distribution between specified minimum and maximum values
    "Log": gf.Distribution(min_=1.0, max_=10.0, type_=gf.DistributionType.LOG),

    # Power-of-two distribution within a range, producing powers of two
    "Powers of two": gf.Distribution(min_=2, max_=1024, type_=gf.DistributionType.POW_TWO, dtype=int)
}

With the distribution examples set up in the previous step, we can now proceed to sample from these distributions to produce random variables. This process demonstrates how `gf.Distribution` can be used to generate a variety of random values based on the specified distribution types.

In [ ]:
# Iterate over each distribution type and sample values from them
for name, distribution in distribution_examples.items():
    # Sample 5 values from each distribution and print them
    print(f"{name} distribution samples: {distribution.sample(5)}")

## Using Distributions to Generate Waveforms

Having demonstrated how to sample from various distributions, we will now apply these concepts to generate gravitational waveforms with randomized parameters. This approach is particularly useful for creating diverse datasets that reflect a wide range of possible astrophysical scenarios.

We will create distributions for key waveform parameters: the masses of the two objects in a binary system (`mass_1_msun` and `mass_2_msun`), and the inclination of the source (`inclination_radians`). These distributions will be used to randomize the parameters of each generated waveform, ensuring variability in our simulations.

The distributions for each parameter are defined as uniform distributions within specified ranges. This uniformity ensures that all values within the range have an equal probability of being selected. Let's proceed to define these distributions and use them to initialize a `gf.cuPhenomDGenerator`, which will generate IMRPhenomD waveforms with these randomized parameters:


In [ ]:
# Define a uniform distribution for the mass of the first object in solar masses.
mass_1_distribution_msun: gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the mass of the second object in solar masses.
mass_2_distribution_msun: gf.Distribution = gf.Distribution(
    min_=10.0, 
    max_=60.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the inclination of the binary system in radians.
inclination_distribution_radians: gf.Distribution = gf.Distribution(
    min_=0.0, 
    max_=np.pi, 
    type_=gf.DistributionType.UNIFORM
)

# Initialize a PhenomD waveform generator with the defined distributions.
# This generator will produce waveforms with randomly varied masses and inclination angles.
phenom_d_distribution_generator: gf.WaveformGenerator = gf.cuPhenomDGenerator(
    mass_1_msun=mass_1_distribution_msun,
    mass_2_msun=mass_2_distribution_msun,
    inclination_radians=inclination_distribution_radians
)

We now utilize the `phenom_d_distribution_generator` to produce waveform injections with varied parameters. By setting up an `gf.InjectionGenerator`, we can create injections where key waveform parameters like masses and inclination angles are randomized. We will also retrieve specific waveform parameters alongside each injection, allowing us to analyze the effect of these variable parameters on the waveforms.

In [ ]:
# Initialize an InjectionGenerator with the PhenomD distribution generator.
# This generator will create waveform injections and will return the specified parameters.
phenom_d_distribution_injection_generator: gf.InjectionGenerator = gf.InjectionGenerator(
    phenom_d_distribution_generator,
    parameters_to_return=[
        gf.WaveformParameters.MASS_1_MSUN, 
        gf.WaveformParameters.MASS_2_MSUN,
        gf.WaveformParameters.INCLINATION_RADIANS
    ]
)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    # Generate a batch of waveform injections with their parameters.
    # num_examples_per_batch is set to 4 to produce four examples.
    phenom_d_distributed_injections, _, phenom_d_distributed_parameters = next(
        phenom_d_distribution_injection_generator(num_examples_per_batch=4)
    )


Next, we extract key parameters such as the masses of the binary companions and their orbital inclination from our generated PhenomD injections. This extraction allows us to observe the impact of these parameters on the waveform characteristics. We will create strain plots for each injection, highlighting the variations in waveform features due to the randomized parameters.

In [ ]:
# Extract the parameters from the distributed PhenomD injections.
mass_1_parameters: tf.Tensor = phenom_d_distributed_parameters[gf.WaveformParameters.MASS_1_MSUN]
mass_2_parameters: tf.Tensor = phenom_d_distributed_parameters[gf.WaveformParameters.MASS_2_MSUN]
inclination_parameters: tf.Tensor = phenom_d_distributed_parameters[gf.WaveformParameters.INCLINATION_RADIANS]

# Initialize an empty layout for the strain plots.
distributed_phenom_layout: List = []

# Iterate over the injections and their corresponding parameters.
for injection, mass_1, mass_2, inclination in zip(
        phenom_d_distributed_injections[0], 
        mass_1_parameters[0], 
        mass_2_parameters[0], 
        inclination_parameters[0]
    ):
    # Create strain plots for each injection with titles displaying the parameter values.
    distributed_phenom_layout.append([
        gf.generate_strain_plot(
            {
                "Plus Polarisation": injection[0],
                "Cross Polarisation": injection[1]
            },
            height=400,
            title=(
                "Phenom D Injection: \n"
                f"Companion 1 Mass: {mass_1:.2f} solar masses.\n"
                f"Companion 2 Mass: {mass_2:.2f} solar masses.\n"
                f"Inclination: {inclination:.2f} Radians."
            )
        )
    ])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(distributed_phenom_layout)
output_notebook()
show(grid)

## Generating Multiple Injections Simultaneously

GravyFlow enables the simultaneous generation of multiple types of injections using a single injection generator. This functionality is particularly beneficial when creating training datasets that require a mix of different injection types, adding variety and complexity to the data.

Let's begin by defining a randomly distributed White Noise Burst (WNB) waveform, similar to how we previously set up the PhenomD waveforms. By incorporating randomness in the waveform parameters, we can simulate a more realistic and varied set of gravitational wave signals.

In [ ]:
# Define a uniform distribution for the duration of the WNB in seconds.
duration_distribution_seconds: gf.Distribution = gf.Distribution(
    min_=0.1, 
    max_=0.9, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the minimum frequency of the WNB in Hertz.
min_frequency_distribution_hertz: gf.Distribution = gf.Distribution(
    min_=20.0, 
    max_=512.0, 
    type_=gf.DistributionType.UNIFORM
)

# Define a uniform distribution for the maximum frequency of the WNB in Hertz.
max_frequency_distribution_hertz: gf.Distribution = gf.Distribution(
    min_=20.0, 
    max_=512.0, 
    type_=gf.DistributionType.UNIFORM
)

# Initialize a WNB waveform generator using the defined distributions.
# This generator will create WNB waveforms with randomized duration and frequency parameters.
wnb_distribution_generator: gf.WaveformGenerator = gf.WNBGenerator(
    duration_seconds=duration_distribution_seconds,
    min_frequency_hertz=min_frequency_distribution_hertz,
    max_frequency_hertz=max_frequency_distribution_hertz
)

To enhance the diversity of our generated dataset, we will now combine both the IMRPhenomD and White Noise Burst (WNB) waveform generators into a single `gf.InjectionGenerator`. This approach allows us to simultaneously generate a variety of waveform injections, each with its distinct characteristics. By doing so, we can create a more comprehensive and varied dataset, suitable for training robust gravitational wave detection models.

We'll specify a list of parameters to be returned with each injection, encompassing key attributes from both PhenomD and WNB waveforms. For consistency, parameters for each waveform will contain entries for each entered type. If they are not used by that particular waveform, they will be set to zero. Let's proceed to set up this multi-type injection generator:


In [ ]:
# Initialize an InjectionGenerator that combines both PhenomD and WNB generators.
# This allows for the generation of a mix of multiple waveform injections.
multi_injection_generator: gf.InjectionGenerator = gf.InjectionGenerator(
    [phenom_d_distribution_generator, wnb_distribution_generator],  # List of waveform generators.
    parameters_to_return=[
        gf.WaveformParameters.MASS_1_MSUN,           # Mass of the first object in solar masses.
        gf.WaveformParameters.MASS_2_MSUN,           # Mass of the second object in solar masses.
        gf.WaveformParameters.INCLINATION_RADIANS,   # Inclination of the binary system.
        gf.WaveformParameters.DURATION_SECONDS,      # Duration of the WNB in seconds.
        gf.WaveformParameters.MIN_FREQUENCY_HERTZ,   # Minimum frequency of the WNB in Hertz.
        gf.WaveformParameters.MAX_FREQUENCY_HERTZ    # Maximum frequency of the WNB in Hertz.
    ]
)


We can then generate multiple injections at each iteration:

In [ ]:
# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    # Generate a batch of multiple types of injections using the multi_injection_generator.
    # num_examples_per_batch is set to 4 to produce four examples in the batch.
    multi_injections, _, multi_parameters = next(multi_injection_generator(num_examples_per_batch=4))

We can plot these waveforms, and their associated parameters, side by side:

In [ ]:
# Extract the parameters from the multi-injections.
mass_1_parameters: tf.Tensor = multi_parameters[gf.WaveformParameters.MASS_1_MSUN][0]
mass_2_parameters: tf.Tensor = multi_parameters[gf.WaveformParameters.MASS_2_MSUN][0]
inclination_parameters: tf.Tensor = multi_parameters[gf.WaveformParameters.INCLINATION_RADIANS][0]
duration_parameters: tf.Tensor = multi_parameters[gf.WaveformParameters.DURATION_SECONDS][1]
min_frequency_parameters: tf.Tensor = multi_parameters[gf.WaveformParameters.MIN_FREQUENCY_HERTZ][1]
max_frequency_parameters: tf.Tensor = multi_parameters[gf.WaveformParameters.MAX_FREQUENCY_HERTZ][1]

# Initialize an empty layout for the strain plots.
distributed_phenom_layout: List = []

# Iterate over the multi-injections and their corresponding parameters.
for phenom_d_injections, wnb_injections, mass_1, mass_2, inclination, duration, min_freq, max_freq in zip(
        multi_injections[0], 
        multi_injections[1],
        mass_1_parameters, 
        mass_2_parameters, 
        inclination_parameters,
        duration_parameters,
        min_frequency_parameters,
        max_frequency_parameters
    ):
    # Create strain plots for each Phenom D and WNB injection with titles displaying the parameter values.
    distributed_phenom_layout.append([
        gf.generate_strain_plot(
            {
                "Plus Polarisation": phenom_d_injections[0],
                "Cross Polarisation": phenom_d_injections[1]
            },
            height=400,
            width=500,
            title=(
                "Phenom D Injection:\n"
                f"Companion 1 Mass: {mass_1:.2f} solar masses.\n"
                f"Companion 2 Mass: {mass_2:.2f} solar masses.\n"
                f"Inclination: {inclination:.2f} Radians."
            )
        ),
        gf.generate_strain_plot(
            {
                "Plus Polarisation": wnb_injections[0],
                "Cross Polarisation": wnb_injections[1]
            },
            height=400,
            width=500,
            title=(
                "WNB Injection:\n"
                f"Duration: {duration:.2f} s.\n"
                f"Minimum Frequency: {min_freq:.2f} Hz.\n"
                f"Maximum Frequency: {max_freq:.2f} Hz."
            )
        )
    ])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(distributed_phenom_layout)
output_notebook()
show(grid)

## Loading Injections from Config JSON

Rather than inputing arguments defined in Python, we can also load injection configurations from a JSON file.

In [ ]:
injection_config_path : Path = Path("./example_configs/phenom_d_parameters.json")

# Load injection config:
loaded_phenom_d_generator : gf.cuPhenomDGenerator = gf.WaveformGenerator.load(injection_config_path)

# Though not pretty, we can print the results to see that the generator has loaded from the json.
print(loaded_phenom_d_generator)

In [ ]:
# Initialize an InjectionGenerator that combines both PhenomD and WNB generators.
# This allows for the generation of a mix of multiple waveform injections.
loaded_injection_generator: gf.InjectionGenerator = gf.InjectionGenerator(
    loaded_phenom_d_generator,  # List of waveform generators.
    parameters_to_return=[
        gf.WaveformParameters.MASS_1_MSUN,           # Mass of the first object in solar masses.
        gf.WaveformParameters.MASS_2_MSUN,           # Mass of the second object in solar masses.
        gf.WaveformParameters.INCLINATION_RADIANS,   # Inclination of the binary system.
    ]
)

# Use the TensorFlow environment 'env' created earlier with gf.env()
with env:
    # Generate a batch of multiple types of injections using the multi_injection_generator.
    # num_examples_per_batch is set to 4 to produce four examples in the batch.
    phenom_d_loaded_injections, _, phenom_d_loaded_parameters = next(loaded_injection_generator(num_examples_per_batch=4))

In [ ]:
# Extract the parameters from the distributed PhenomD injections.
mass_1_parameters: tf.Tensor = phenom_d_loaded_parameters[gf.WaveformParameters.MASS_1_MSUN]
mass_2_parameters: tf.Tensor = phenom_d_loaded_parameters[gf.WaveformParameters.MASS_2_MSUN]
inclination_parameters: tf.Tensor = phenom_d_loaded_parameters[gf.WaveformParameters.INCLINATION_RADIANS]

# Initialize an empty layout for the strain plots.
loaded_phenom_layout: List = []

# Iterate over the injections and their corresponding parameters.
for injection, mass_1, mass_2, inclination in zip(
        phenom_d_loaded_injections[0], 
        mass_1_parameters[0], 
        mass_2_parameters[0], 
        inclination_parameters[0]
    ):
    # Create strain plots for each injection with titles displaying the parameter values.
    loaded_phenom_layout.append([
        gf.generate_strain_plot(
            {
                "Plus Polarisation": injection[0],
                "Cross Polarisation": injection[1]
            },
            height=400,
            title=(
                "Phenom D Injection: \n"
                f"Companion 1 Mass: {mass_1:.2f} solar masses.\n"
                f"Companion 2 Mass: {mass_2:.2f} solar masses.\n"
                f"Inclination: {inclination:.2f} Radians."
            )
        )
    ])

# Arrange the plots in a grid layout and display them in the notebook.
grid = gridplot(loaded_phenom_layout)
output_notebook()
show(grid)

## Varying Injection Chance